<a href="https://colab.research.google.com/github/ShykerBogdan/Prozzoro_project/blob/master/DataPrepr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import time
from datetime import datetime
import numpy as np

In [6]:
filename='/content/gdrive/My Drive/ML_proz/data_source.csv'
df=pd.read_csv(filename)
df.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5,14,15,16,17,18,20,24,38,48,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3507778, 56)

In [0]:



class DataPreproccessing:
    def __init__(self, data_source, churn_interval, start_period=None, end_period=None):
        self.last_day=pd.to_datetime('2018-11-29')
        # для статистики постачальника вибираємо проміжок [start_period,end_period]
        self.divide_data = self.last_day - pd.DateOffset(days=churn_interval)
        print(self.divide_data)
        self.data_source = data_source[pd.to_datetime(data_source['date_from_id']) <= self.divide_data]
        # для labeling вибираємо період [end_period, end_period+churn_interval]
        self.label_data = data_source[pd.to_datetime(data_source['date_from_id']) > self.divide_data]
        # вибираємо унікальних постачальників
        self.unique_id = list(self.data_source['participants'].value_counts().index)
        # створюємо dataframe з колонкою унікальних постачальників і туди будемо додавати features
        self.feature_data = pd.DataFrame(self.unique_id, columns=['unique_id'])
        print(self.feature_data.iloc[:10]['unique_id'].values)
     
    def cleaning(self):
        self.data_source['winner']=self.data_source[['contracts.value_amount','contracts.status','winner']].apply(lambda x: 0 if x['contracts.value_amount']=='\\N'
                                                                                                         or x['contracts.status']=='cancelled' else x['winner'],axis=1  )
        print('winner updated')
        self.activity_date()
        print('activity date')
    def split_into_batches(self,n=20000,filename=None):
        self.cleaning()
        feature_batches=list()
        n_iter=int(np.ceil(self.feature_data.shape[0]/n))
        data=self.data_source[['participants','new_date']]
        for i in range(n_iter):
          print('Batch {0}-{1}:'.format(i*n,(i+1)*n))
          unique_id=self.feature_data.iloc[i*n:(i+1)*n]['unique_id'].values
          feature_df=pd.DataFrame(unique_id,columns=['unique_id'])
          feature_batches.append(feature_df)
          # part of data_source
          df_=data.loc[data['participants'].isin(unique_id)]
          print('Unique_supp:{0}-{1}'.format(len(unique_id),df_.shape[0]))
          df_=df_.sort_values(by='new_date')
          def get_average_activity(raw,data):
            dates_=list(df_[df_['participants']==raw['unique_id']]['new_date'].values)
            try:
              raw['average_activity']=(pd.to_datetime(dates_[-1])-pd.to_datetime(dates_[0])).days/len(dates_)
            except:
              raw['average_activity']=np.nan
            return raw
          feature_batches[i]=feature_batches[i].apply(get_average_activity,args=(df_,),axis=1)
          if filename:
            print(f'write to csv')
            feature_batches[i].to_csv(filename+ '{0}'.format(i)+'.csv',index=False)
        # concat dataframes into one
        print('Concat')
        all_batches_df = pd.concat(feature_batches)
        if filename:
          print('write to csv_all')
          all_batches_df.to_csv(filename+'_all.csv',index=False)
          
          
        
    
    def get_label(self):
        participants=list(self.label_data['participants'].value_counts().index)
        
        print(len(participants))
        def get_y(raw,participants):
          if raw['unique_id'] in participants:
            raw['y']=1
          else:
            raw['y']=0
          return raw
        
        self.feature_data=self.feature_data.apply(get_y,args=(participants,),axis=1)
        
    def graph_2(self,filename=None):
        self.data_source['winner']=self.data_source[['contracts.value_amount','contracts.status','winner']].apply(lambda x: 0 if x['contracts.value_amount']=='\\N'
                                                                                                         or x['contracts.status']=='cancelled' else x['winner'],axis=1  )

        self.activity_date()
        print('activity date')
        
        data=self.data_source[['participants','new_date']]
        data=data.sort_values(by='new_date')
        def get_average_activity(raw,data):
          dates_=list(data[data['participants']==raw['unique_id']]['new_date'].values)
          try:
            raw['average_activity']=(pd.to_datetime(dates_[-1])-pd.to_datetime(dates_[0])).days/len(dates_)
          except:
            raw['average_activity']=np.nan
          return raw
        
        self.feature_data=self.feature_data.apply(get_average_activity,args=(data,),axis=1)
        print('average_activity')
        if filename:
          print('write to csv')
          self.feature_data.to_csv(filename)
    
    def create_features(self,filename=None):
        self.get_label()
        print('get y')
        self.data_source['winner']=self.data_source[['contracts.value_amount','contracts.status','winner']].apply(lambda x: 0 if x['contracts.value_amount']=='\\N'
                                                                                                         or x['contracts.status']=='cancelled' else x['winner'],axis=1  )
        print('winner updated')
        self.activity_date()
        print('activity date')
        
#         self.average_economy()
#         print('average economy')
        self.count_lots()
        print('count_lots has already finish')
        self.count_win_lose()
        print('count_win_lose has already finish')
#         self.count_win_type_procedure()
#         print('count_win_type_procedure has already finish')
#         self.count_lose_type_procedure()
#         print('count_lose_type_procedure has already finish')
#         self.feature_data['count_lots_open']=self.feature_data['win_open']+self.feature_data['lose_open']
#         self.feature_data['count_lots_not_open']=self.feature_data['win_not_open']+self.feature_data['lose_not_open']
#         print('count_lots_open has already finish')
        self.last_activity()
        print('last activity')
#         self.average_activity()
#         print('average activity')
        
        
        if filename:
          self.feature_data.to_csv(filename)
          
    def count_lots(self):
        freq = self.data_source['participants'].value_counts()

        def count_lots(raw, frequency):
            raw['count_lots'] = frequency[raw['unique_id']]
            return raw

        self.feature_data = self.feature_data.apply(count_lots, args=(freq,), axis=1)

    def count_win_lose(self):
        data=self.data_source[self.data_source['winner']==1]
        frequency=data['participants'].value_counts()
        
        def win_lose(raw,frequency):
            # use try bsc if index not in frequency 'win'=0
            try:
              raw['win']=frequency[raw['unique_id']]
            except:
              raw['win']=0
            return raw
            

        self.feature_data = self.feature_data.apply(win_lose, args=(
        frequency,),axis=1)
        self.feature_data['lose']=self.feature_data['count_lots']-self.feature_data['win']
        
    # всьо в одну функцію
    def count_win_type_procedure(self):
        data=self.data_source[self.data_source['winner']==1]
        data=data[data['tenders.procurementMethod']=='open']
        frequency=data['participants'].value_counts()
        def count_win_open(raw,frequency):
            try:
              raw['win_open']=frequency[raw['unique_id']]
            except:
              raw['win_open']=0
            return raw
              
        self.feature_data = self.feature_data.apply(count_win_open, args=(
        frequency,),axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))
        self.feature_data['win_not_open']=self.feature_data['win']-self.feature_data['win_open']
    
    
    def count_lose_type_procedure(self):
        data=self.data_source[self.data_source['winner']==0]
        data=data[data['tenders.procurementMethod']=='open']
        frequency=data['participants'].value_counts()
        def count_lose_open(raw,frequency):
            try:
              raw['lose_open']=frequency[raw['unique_id']]
            except:
              raw['lose_open']=0
            return raw
              
        self.feature_data = self.feature_data.apply(count_lose_open, args=(
        frequency,),axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))
        self.feature_data['lose_not_open']=self.feature_data['lose']-self.feature_data['lose_open']
    
    
    def activity_date(self):
        f=['winner','tenders.auctionPeriod_startDate','contracts.dateSigned','date_from_id','awards.date']
        self.data_source['new_date']=self.data_source[f].apply(lambda x:x[f[2]] if x[f[0]==1] else (x[f[3]] if pd.isnull(x[f[1]]) else x[f[1]] ),axis=1)
        self.data_source['new_date']=self.data_source[['new_date']].apply(lambda x: x['new_date'][:10],axis=1)
        self.data_source['new_date']=self.data_source[['new_date','date_from_id']].apply(lambda x: x['date_from_id'] if x['new_date']=='\\N' else x['new_date'],axis=1)
        
        
    def last_activity(self):
        data=self.data_source[['participants','new_date']]
        data=data.sort_values(by='new_date')
        self.divide_data=str(self.divide_data)[:10]
        
        def get_last_activity(raw,data):
          raw['last_activity_date']=data[data['participants']==raw['unique_id']].iloc[-1]['new_date']
          try:
            raw['last_activity_days']=(pd.to_datetime(self.divide_data)-pd.to_datetime(raw['last_activity_date'])).days
          except:
            print('--------------')
            raw['last_activity_days']='?'
          return raw
        
        
        self.feature_data=self.feature_data.apply(get_last_activity,args=(data,),axis=1)
        self.feature_data['last_activity_days']=self.feature_data.apply(lambda x: 0 if x['last_activity_days']<0 else x['last_activity_days'],axis=1)
        
    
    
    def average_activity(self):
        data=self.data_source[['participants','new_date']]
        data=data.sort_values(by='new_date')
        def get_average_activity(raw,data):
          dates=list(data[data['participants']==raw['unique_id']]['new_date'].values)
#           dates=np.diff([pd.to_datetime(item) for item in dates])

#           dates=[item.days for item in dates]
#           dates.append(raw['last_activity_days'])
          raw['average_activity']=np.average(dates)
          return raw
    
        self.feature_data=self.feature_data.apply(get_average_activity,args=(data,),axis=1)
     
    def average_economy(self):
        def convert_to_float(raw):
          try:
            raw['lots.value_amount']=float(raw['lots.value_amount'])
          except:
            raw['lots.value_amount']=np.nan
          return raw
        self.data_source['lots.value_amount']=self.data_source[['lots.value_amount']].apply(convert_to_float,axis=1)
        print('convert lots')
        data=self.data_source[self.data_source['winner']==1][['participants','awards.value_amount','lots.value_amount']]
        data['economy_value']=data['lots.value_amount']-data['awards.value_amount']
        data['economy_percent']=data['awards.value_amount']/data['lots.value_amount']
        def get_economy(raw,data):
          d=data[data['participants']==raw['unique_id']]
          raw['economy_value']=np.nanmean(d['economy_value'])
          raw['economy_percent']=np.nanmean(d['economy_percent'])
#           raw['economy_value']=np.mean(d['lots.value_amount']-d['awards.value_amount'])
#           raw['economy_percent']=np.mean(d['awards.value_amount']/d['lots.value_amount'])
          return raw
        self.feature_data=self.feature_data.apply(get_economy,args=(data,),axis=1)
          
        

        
        
        
        


In [0]:
col=['winner','tenders.auctionPeriod_startDate','contracts.dateSigned',
     'awards.date','participants','date_from_id','contracts.value_amount','contracts.status']

In [23]:
start_time = time.time()
df1=df[col]
prepr = DataPreproccessing(df1,1)
prepr.split_into_batches(n=10000,filename='try')
print("--- %s seconds ---" % (time.time() - start_time))
print(prepr.feature_data.head())
# 158 sec - 30000

2018-11-28 00:00:00
['25394112' '32490244' '39273420' '41449359' '40473930' '36248687'
 '21633086' '21560766' '39190161' '33680859']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


winner updated


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

activity date
Batch 0-10000:
Unique_supp:10000-2021555
write to csv
Batch 10000-20000:
Unique_supp:10000-435477
write to csv
Batch 20000-30000:
Unique_supp:10000-259041
write to csv
Batch 30000-40000:
Unique_supp:10000-174904
write to csv
Batch 40000-50000:
Unique_supp:10000-126328
write to csv
Batch 50000-60000:
Unique_supp:10000-95018
write to csv
Batch 60000-70000:
Unique_supp:10000-73963
write to csv
Batch 70000-80000:
Unique_supp:10000-57168
write to csv
Batch 80000-90000:
Unique_supp:10000-46246
write to csv
Batch 90000-100000:
Unique_supp:10000-38369
write to csv
Batch 100000-110000:
Unique_supp:10000-30000
write to csv
Batch 110000-120000:
Unique_supp:10000-25296
write to csv
Batch 120000-130000:
Unique_supp:10000-20000
write to csv
Batch 130000-140000:
Unique_supp:10000-20000
write to csv
Batch 140000-150000:
Unique_supp:10000-14544
write to csv
Batch 150000-160000:
Unique_supp:10000-10000
write to csv
Batch 160000-170000:
Unique_supp:10000-10000
write to csv
Batch 170000-1800